<a href="https://colab.research.google.com/github/R329aghav/Raghavjr/blob/main/Air_Quality_Index_Prediction_Using_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras
from keras.layers import Dense
from keras.models import Sequential
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch
path="/content/drive/MyDrive/DATASETS/Air Quality Index- Delhi.csv"
data=pd.read_csv(path)
data


,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
726,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
727,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
728,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
729,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [ ]:
X=data.drop(['PM 2.5'],axis=1)
y=data['PM 2.5']

In [ ]:
def build_model(hp):
  model=keras.Sequential()
  for i in range(hp.Int('num_layers',2,20)):
    model.add(layers.Dense(units=hp.Int('units_'+str(i),
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation="relu"))
    model.add(layers.Dense(1,activation="linear"))
    model.compile(optimizer=tf.keras.optimizers.Adam(
        hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model


In [ ]:
tuner=RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='DATASETS',
    project_name='Air Quality Index-Delhi',
)


INFO:tensorflow:Reloading Oracle from existing project DATASETS/Air Quality Index-Delhi/oracle.json
INFO:tensorflow:Reloading Tuner from DATASETS/Air Quality Index-Delhi/tuner0.json


In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
tuner.search(X_train,y_train,
             epochs=5,
             validation_data=(X_test,y_test))

INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in DATASETS/Air Quality Index-Delhi
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 18
units_0: 160
learning_rate: 0.01
Score: nan
Trial summary
Hyperparameters:
num_layers: 10
units_0: 288
learning_rate: 0.01
Score: nan
Trial summary
Hyperparameters:
num_layers: 9
units_0: 64
learning_rate: 0.001
Score: nan
Trial summary
Hyperparameters:
num_layers: 4
units_0: 192
learning_rate: 0.0001
Score: nan
Trial summary
Hyperparameters:
num_layers: 7
units_0: 352
learning_rate: 0.01
Score: nan
